In [1]:
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from keras.preprocessing.image import load_img, img_to_array 
from flask import Flask, jsonify, request

In [2]:
model = load_model('models/MobNet_model_3classes.h5')

Metal device set to: Apple M1


2023-09-14 13:58:57.936998: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-14 13:58:57.937277: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
def predict_defect(image_path):
    img = load_img(image_path, target_size= (224, 224, 3))
    img = img_to_array(img)
    #img = cv2.imread(image_path)
    #resize = tf.image.resize(img, (224,224))
    yhat = model.predict(np.expand_dims(img/255, 0)).argmax(axis=1)
    if yhat == 0:
        pred = 'crack'
    elif yhat == 1:
        pred = 'pothole'
    else:
        pred = 'rut'
    return pred

In [4]:
#predict_defect('road_images/im_to_test/crack3.jpg')

2023-09-14 13:54:54.052241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 822ms/step


'crack'

In [4]:
app = Flask(__name__)

@app.route('/', methods=['POST'])

def infer_image():
    if 'file' not in request.files:
        return jsonify(error="Please try again. The image doesn't exist")
    
    file = request.files.get('file')
    img_bytes = file.read()
    img_path = "./road_images/im_to_test/crack3.jpg"
    with open(img_path, "wb") as img:
        img.write(img_bytes)
    result = predict_defect(img_path)
    return jsonify(prediction=result)

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Sep/2023 13:59:07] "GET / HTTP/1.1" 405 -
2023-09-14 13:59:16.396458: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 794ms/step


127.0.0.1 - - [14/Sep/2023 13:59:16] "POST / HTTP/1.1" 200 -


1/1 [==============================] - 1s 1s/step


127.0.0.1 - - [14/Sep/2023 14:49:14] "POST / HTTP/1.1" 200 -
